In [3]:
import pandas as pd

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer

In [7]:
!java --version

openjdk 17.0.7 2023-04-18
OpenJDK Runtime Environment (build 17.0.7+7-Ubuntu-0ubuntu120.04)
OpenJDK 64-Bit Server VM (build 17.0.7+7-Ubuntu-0ubuntu120.04, mixed mode, sharing)


In [2]:
spark = sparknlp.start()

:: loading settings :: url = jar:file:/mnt/vol_b/miniconda3/envs/poseapi3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7123477b-55f6-4baa-a383-c64813b6388b;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.2.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

23/12/09 12:56:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
pyspark.version()

TypeError: 'module' object is not callable

In [3]:
num_topics = 15
lda = LDA(k=num_topics, maxIter=10)

In [4]:
df = pd.read_csv("unsupervised_data.csv")
df.head()

,ChannelName,Date,Content
0,krovatka_luki,2023-10-01 13:46:39.000,Ну вот со всех подряд и начинай😁
1,dumerz,2023-10-01 19:25:23.000,да
2,moscowtam,2023-10-01 17:41:32.000,Смотрела званый ужин?
3,uavdevchat,2023-10-01 15:55:10.000,хера тебя штырит. Это рептилоиды.
4,kadyrov_95chat,2023-10-01 12:39:54.000,Мира и процветания нашей Республике


In [5]:
reduced_df = df[:1000]
reduced_df.shape

(1000, 3)

In [7]:
reduced_df.to_csv("un_subset.csv", index=False)

In [78]:
import re
import csv

def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text)

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_text(text):
    text = remove_urls(text)
    text = remove_emojis(text)
    return text.strip()


def multiline_csv_reader(file_path, max_num=100):
    with open(file_path, 'r', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        headers = next(csv_reader)
        
        i=0
        for row in csv_reader:
            
            if i >= max_num:
                break
            text = row[-1]
            text = preprocess_text(text)
            if text:
                i += 1
                yield text

In [79]:
for elm in multiline_csv_reader("un_subset.csv"):
    print(elm)
    break

Ну вот со всех подряд и начинай


In [80]:
sdf = spark.read.format("csv").option("header",True).load("un_subset.csv")

In [81]:
sdf

DataFrame[ChannelName: string, Date: string, Content: string]

In [82]:
sdf.show()

+---------------+--------------------+--------------------+
|    ChannelName|                Date|             Content|
+---------------+--------------------+--------------------+
|  krovatka_luki|2023-10-01 13:46:...|Ну вот со всех по...|
|         dumerz|2023-10-01 19:25:...|                  да|
|      moscowtam|2023-10-01 17:41:...|Смотрела званый у...|
|     uavdevchat|2023-10-01 15:55:...|хера тебя штырит....|
| kadyrov_95chat|2023-10-01 12:39:...|Мира и процветани...|
|screen_and_link|2023-10-01 13:22:...|              Алехин|
| kadyrov_95chat|2023-10-01 12:37:...|            👍 удачи|
|  putin_sobchak|2023-10-01 05:11:...|https://www.youtu...|
|  putin_sobchak|2023-10-01 09:30:...|https://www.youtu...|
|  putin_sobchak|2023-10-01 05:13:...|https://www.youtu...|
|hze7fcbzsdhnzm0|2023-10-01 06:08:...|         Цирк шапито|
|  putin_sobchak|2023-10-01 08:44:...|https://www.youtu...|
|  putin_sobchak|2023-10-01 05:02:...|https://www.youtu...|
|  putin_sobchak|2023-10-01 08:58:...|htt

In [83]:
# sdf = sdf.withColumn("Content", pyspark.sql.functions.array(sdf["Content"]))

In [84]:
document_assembler = DocumentAssembler() \
    .setInputCol("Content") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .pretrained('stopwords_ru', 'ru')\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemmatizer = LemmatizerModel\
    .pretrained("lemma", "ru") \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

nlp_pipeline = Pipeline(
    stages=[
            document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemmatizer,
            finisher
])

stopwords_ru download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
lemma download started this may take some time.
Approximate size to download 1.3 MB
[OK!]


In [85]:
nlp_model = nlp_pipeline.fit(sdf)

In [86]:
nlp_model.stages

[DocumentAssembler_be4637c717ee,
 REGEX_TOKENIZER_63d0ce2e9d70,
 NORMALIZER_a94001f1d001,
 STOPWORDS_CLEANER_3187062cd9db,
 LEMMATIZER_59ad4a12c2e7,
 Finisher_b88cc9c66ab1]

In [87]:
processed_df = nlp_model.transform(sdf)

In [88]:
tokens_df = processed_df.select('tokens').limit(10000)
tokens_df.show()

+--------------------+
|              tokens|
+--------------------+
|  [подряд, начинать]|
|                  []|
|[Смотрела, званый...|
|[хера, штырит, ре...|
|[процветание, наш...|
|            [Алехин]|
|             [удача]|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|      [цирк, шапито]|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
|[httpswwwyoutubec...|
+--------------------+
only showing top 20 rows



In [89]:
tokens_df.select("tokens").take(5)

[Row(tokens=['подряд', 'начинать']),
 Row(tokens=[]),
 Row(tokens=['Смотрела', 'званый', 'ужин']),
 Row(tokens=['хера', 'штырит', 'рептилоиды']),
 Row(tokens=['процветание', 'нашей', 'Республика'])]

In [91]:
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=10000, minDF=3.0)
cv_model = cv.fit(tokens_df)
vectorized_tokens = cv_model.transform(tokens_df)
vectorized_tokens

DataFrame[tokens: array<string>, features: vector]

In [92]:
num_topics = 15
lda = LDA(k=num_topics, maxIter=10)

In [93]:
%%time
model = lda.fit(vectorized_tokens)

CPU times: user 15.2 ms, sys: 4.19 ms, total: 19.4 ms
Wall time: 2.25 s


In [94]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

vocab = cv_model.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd \
       .map(lambda row: row['termIndices']) \
       .map(lambda idx_list: [vocab[idx] for idx in idx_list]) \
       .collect()

def get_words(idx_list):
    return [vocab[idx] for idx in idx_list]

udf_get_words = udf(get_words, ArrayType(StringType()))
topics = topics.withColumn("words", udf_get_words(topics.termIndices))

topics_df = topics.select("topic", "words")

topics_df.show(truncate=False)

+-----+----------------------------------------------------------------------------------------+
|topic|words                                                                                   |
+-----+----------------------------------------------------------------------------------------+
|0    |[защитник, воинам, позиция, Успех, the, тема, of, that, страна, получиться]             |
|1    |[воздух, буква, позиция, дело, идти, войла, воинам, TRANSLATION, стараться, боевой]     |
|2    |[Украина, РФ, работать, нужный, стать, делать, хьо, судить, победить, выполнение]       |
|3    |[писать, говорить, походу, reklama, iltimos, tarqatmang, регион, хороший, цена, поднять]|
|4    |[ночь, пусть, благой, хороший, мусульманин, быстро, примета, Аллах, новость, дело]      |
|5    |[живой, and, получать, объект, is, наблюдать, идти, готовый, интересный, главное]       |
|6    |[Дала, йойла, беркате, бойла, бойцам, удача, наш, таьхье, вай, желать]                  |
|7    |[канал, никто, группа, 